# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [99]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,11.05,70,0,1.54,US,1682555038
1,1,utrik,11.2278,169.8474,26.71,81,42,9.06,MH,1682555136
2,2,port lincoln,-34.7333,135.8667,17.96,80,12,8.69,AU,1682555137
3,3,puerto natales,-51.7236,-72.4875,5.11,84,73,2.51,CL,1682555137
4,4,pacific grove,36.6177,-121.9166,24.80,79,0,2.24,US,1682555137


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display
# Configure the map
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.6,
    color = "City"
)

# Display the map plot
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [92]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df1 = city_data_df.dropna(how='any')

# Drop any rows with null values
city_good_weather_df = city_data_df1.loc[(city_data_df1['Max Temp']>21)&(city_data_df1['Max Temp']<27)&(city_data_df1['Wind Speed']<4.5)&(city_data_df1['Cloudiness']<20)]

# Display sample data
city_good_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,pacific grove,36.6177,-121.9166,24.80,79,0,2.24,US,1682555137
21,21,adamstown,-25.0660,-130.1015,24.29,66,8,3.99,PN,1682555141
36,36,puerto santa,-8.9906,-78.6456,24.48,82,16,4.07,PE,1682555145
65,65,belmonte,-15.8631,-38.8828,24.42,85,5,1.40,BR,1682555153
69,69,inhambane,-23.8650,35.3833,21.83,83,6,2.68,MZ,1682555154
78,78,tura,25.5198,90.2201,22.96,47,6,1.25,IN,1682555156
81,81,medina,24.4686,39.6142,25.24,22,0,2.06,SA,1682554913
83,83,jinghong,21.9910,100.7341,22.51,69,1,0.44,CN,1682555157
97,97,nova vicosa,-17.8919,-39.3719,24.47,85,4,2.32,BR,1682555161
135,135,awjilah,29.1081,21.2869,22.67,27,0,2.08,LY,1682555172


### Step 3: Create a new DataFrame called `hotel_df`.

In [107]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = pd.DataFrame({"City":city_good_weather_df["City"],
                         "Country":city_good_weather_df["Country"],
                         "Latitude":city_good_weather_df["Lat"],
                         "Longitude":city_good_weather_df["Lng"],
                         "Humidity":city_good_weather_df["Humidity"],})


# Display sample data
hotel_df

,City,Country,Latitude,Longitude,Humidity
4,pacific grove,US,36.6177,-121.9166,79
21,adamstown,PN,-25.0660,-130.1015,66
36,puerto santa,PE,-8.9906,-78.6456,82
65,belmonte,BR,-15.8631,-38.8828,85
69,inhambane,MZ,-23.8650,35.3833,83
78,tura,IN,25.5198,90.2201,47
81,medina,SA,24.4686,39.6142,22
83,jinghong,CN,21.9910,100.7341,69
97,nova vicosa,BR,-17.8919,-39.3719,85
135,awjilah,LY,29.1081,21.2869,27


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [110]:
# Set parameters to search for a hotel
radius = 100000
limit = 20


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index,"Latitude"]
    lng = hotel_df.loc[index,"Longitude"]
    filters =f"circle:{lat},{lng},{radius}"
    bias = f"proximity:{lat},{lng}"
    categories ="accommodation.hotel"
    
 
    # Set base URL
    qurl=f"https://api.geoapify.com/v2/places?categories={categories}&filter={filters}&limit=20&apiKey={geoapify_key}"


    # Make and API request using the params dictionaty
    name_response = requests.get(qurl)
    
    # Convert the API response to JSON format
    name_address = name_response.json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
pacific grove - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
puerto santa - nearest hotel: No hotel found
belmonte - nearest hotel: No hotel found
inhambane - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
medina - nearest hotel: Thalassa Boutique Hotel
jinghong - nearest hotel: No hotel found
nova vicosa - nearest hotel: No hotel found
awjilah - nearest hotel: No hotel found
bamako - nearest hotel: Aldeamente Turistica Pasargada
dhekiajuli - nearest hotel: No hotel found
puerto baquerizo moreno - nearest hotel: No hotel found
brasileia - nearest hotel: No hotel found
hpa-an - nearest hotel: No hotel found
grand baie - nearest hotel: No hotel found


,City,Country,Latitude,Longitude,Humidity,Hotel Name
4,pacific grove,US,36.6177,-121.9166,79,No hotel found
21,adamstown,PN,-25.0660,-130.1015,66,No hotel found
36,puerto santa,PE,-8.9906,-78.6456,82,No hotel found
65,belmonte,BR,-15.8631,-38.8828,85,No hotel found
69,inhambane,MZ,-23.8650,35.3833,83,No hotel found
78,tura,IN,25.5198,90.2201,47,No hotel found
81,medina,SA,24.4686,39.6142,22,Thalassa Boutique Hotel
83,jinghong,CN,21.9910,100.7341,69,No hotel found
97,nova vicosa,BR,-17.8919,-39.3719,85,No hotel found
135,awjilah,LY,29.1081,21.2869,27,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [109]:
%%capture --no-display
# Configure the map
map_plot2 = hotel_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.6,
    color = "City",
    hover_cols=["City","Hotel Name"])

# Display the map plot
map_plot2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity,Hotel Name)